# SAM 2024 : TP Requêtes multi-sources pour l'open data

Séances 8 et 9

date du document : 28/03/2024


Objectif : Comprendre et réaliser des jointures entre deux sources réelles distantes lorsque l'accès aux sources est conforme aux standards de l'*open data*.  

Contexte : On considère deux sources réelles ainsi que des exemples de requêtes pouvant être posées sur chacune d'entre elles.

On vous demande de mettre en oeuvre des scénarios de requêtes **MULTI** sources.

POur 2024 : Faire l'exercice 3 en priorité




## Préparation

**Copier** ce notebook dans votre drive et l'exécuter dans l'environnement colab de google


In [1]:
!pip install SPARQLWrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.5 MB/s eta 0:00:00


### Manipuler le résultat d'une requête
Fonctions **tableauResultat** pour extraire  le résultat d'une requête

In [2]:
import pandas as pd

#pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

# Déterminer la liste des noms d'attributs du résultat, en tenant compte des attributs optionnels.
def schemaResultat(resultats, limit=100):
  d = {}
  for elt in resultats[:limit]:
    for (pos, nom) in enumerate(elt):
      sum_pos, nb = d.get(nom, (0, 0))
      d[nom] = (sum_pos + pos, nb + 1)
  mean_pos_by_name = [(sum_pos / nb, nom) for (nom, (sum_pos, nb)) in d.items()]
  colonnes = [nom for (mean_pos, nom) in sorted(mean_pos_by_name)]
  return colonnes

def tableauResultat(resultats, limit=100, colonnes=None):
  if colonnes is None :
    colonnes = schemaResultat(resultats, limit)
  tableau = []
  for elt in resultats[:limit]:
    row = {name:v.value for name, v in elt.items()}
    tableau.append([row.get(n, "") for n in colonnes])
  return tableau

print ("fonctions définies")

fonctions définies


Afficher le résultat

In [3]:
def afficherResultat(resultats, limit=100):
  print(len(resultats), "éléments")
  colonnes = schemaResultat(resultats, limit)
  print("schéma: ", colonnes)
  result = tableauResultat(resultats, limit, colonnes)
  return pd.DataFrame(result, columns = colonnes)

print ("fonctions définies")

fonctions définies


### Importer le pilote pour accéder aux sources

In [4]:
from SPARQLWrapper import SPARQLWrapper2
print("pilote importé")

pilote importé


### Préparer l'accès à la source BNF

In [5]:
# Connexion à la BNF
BNF = SPARQLWrapper2("http://data.bnf.fr/sparql")

BNF_prefix = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rda: <http://rdaregistry.info/Elements/w/#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX bnf: <http://data.bnf.fr/ontology/bnf-onto/>

PREFIX bio: <http://vocab.org/bio/0.1/>
PREFIX voc2: <http://rdvocab.info/ElementsGr2/>
"""
print(BNF_prefix)


PREFIX dc: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rda: <http://rdaregistry.info/Elements/w/#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX bnf: <http://data.bnf.fr/ontology/bnf-onto/>

PREFIX bio: <http://vocab.org/bio/0.1/>
PREFIX voc2: <http://rdvocab.info/ElementsGr2/>



### Préparer l'accès à la source DBPEDIA

In [6]:
# Connexion à DBPEDIA France.
DBPEDIA = SPARQLWrapper2("http://fr.dbpedia.org/sparql")

DBPEDIA_prefix =  """
PREFIX dc:   <http://purl.org/dc/terms/>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

PREFIX dbo:  <http://dbpedia.org/ontology/>

# Dans cette source, les noms de propriétés sont en français
PREFIX dbp:  <http://fr.dbpedia.org/property/>
"""
# print(DBPEDIA_prefix)

## Source 1 : La bibliothèque BNF

### BNF P1: Le nom des personnes
On suppose les relations :
*    **CrééPar**(ouvrage, auteur)
*    **Nom**(auteur, nom)

En SQL, la requête s'exprimerait ainsi:

> select distinct n.nom

> from CrééPar c, Nom n

> where c.auteur = n.auteur

In [7]:
requete = BNF_prefix + """
SELECT distinct ?nom
WHERE {
  ?ouvrage dc:creator ?auteur .       # un ouvrage est créé par au moins un auteur
  ?auteur foaf:name ?nom              # un auteur a un nom
}
LIMIT 5
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
for r in tableauResultat(resultats):
  print(r)

['Claude Baival']
['Musée national du Château de Versailles et de Trianon']
['Henry Bordeaux']
['Paul Jungers']
["École nationale supérieure d'art. Dijon"]


### BNF T1: liste de thèmes
#### Les *themes* des oeuvres dont l'auteur est référencé dans dbpedia france

En SQL, cette requête pourrait s'exprimer sur les tables suivantes

**créé_par** (livre, auteur)

**type_d_expression** (livre, type_expre)  -- exple de tpye d'expression: textuelle

**écrit_en** (livre, langue)

**sujet** (livre, thème)

**auteur_équivalent** (auteur, auteur_source2)

et une condition SQL serait : WHERE auteur_source2 like 'http://fr.dbpedia.org%'

In [8]:
requete = BNF_prefix + """
SELECT distinct ?theme
WHERE {

  # une oeuvre est associée a (au moins) un auteur
  ?livre dc:creator ?auteur .

  # un livre est un ouvrage dont la forme d'expression est textuelle (te)
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # livre écrit en langue française
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a au moins un theme
  ?livre bnf:subject ?theme.

  # l'auteur existe également dans une autre source,
  # on dit qu'il est "aligné" avec un autre instance du même individu.
  ?auteur owl:sameAs ?auteurDbpedia .

  # la référence externe "pointe" vers dbpedia
  FILTER ( regex(?auteurDbpedia, "http://fr.dbpedia.org")  )

}
ORDER BY ?theme

LIMIT 20
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)

# Exemple:
# Informatique
# Langues
# Philosophie

20 éléments
schéma:  ['theme']


,theme
0,Administration publique
1,Anthropologie. Ethnologie
2,Architecture
3,Archéologie. Préhistoire. Histoire ancienne
4,Art
5,Art et science militaires
6,Arts du spectacle
7,Arts graphiques
8,Astronomie
9,Audiovisuel


###  BNF A1 : une liste d'auteurs

A1 : Liste des auteurs de livres écrits en français et nés pendant une certaine période, par exemple au 20eme siècle. Afficher le nom de l'auteur, son année et lieu de naissance.



In [9]:
requete = BNF_prefix + """
SELECT distinct ?nom  ?naissance
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom
?auteur foaf:name ?nom .

# un auteur a une?lieu année et un lieu de naissance
?auteur bio:birth ?naissance.
?auteur voc2:placeOfBirth ?lieu

# on sélectionne les années de naissance dans l'intervalle [1900, 2000]
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer
     && xsd:integer (?naissance) < '2000'^^xsd:integer )
}
#ORDER BY ?nom
LIMIT 20
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)

20 éléments
schéma:  ['nom', 'naissance']


,nom,naissance
0,Alex Pol Axely,1920
1,Alexandra Pecker,1906
2,Marie-Françoise Christout,1928
3,Gabriel Cousin,1918
4,Éric Muraise,1908
5,Marc Peyre,1904
6,Jean Savatier,1922
7,Hans Ludwig Cohn Jaffé,1915
8,Jean Canu,1910
9,Albert Ayme,1920


###  BNF A2 : les auteurs de la BNF référencés dans dbpedia
Les auteurs ayant un identifiant externe pointant vers dbpedia


In [10]:
requete = BNF_prefix + """
SELECT distinct ?nom  ?naissance ?auteurDbpedia
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom, et une année de naissance
?auteur foaf:name ?nom ; bio:birth ?naissance .

# l'auteur existe également dans une source externe,
# on dit qu'il est "aligné" avec un autre instance du même individu.
?auteur owl:sameAs ?auteurDbpedia .

# naissance dans l'intervalle [1900, 2000] et la référence externe "pointe" vers dbpedia
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer
     && xsd:integer (?naissance) < '2000'^^xsd:integer
     && regex(?auteurDbpedia, "http://fr.dbpedia.org")
     )
}
ORDER BY desc(?naissance)
LIMIT 10
"""

BNF.setQuery(requete)

resultats = BNF.query().bindings
afficherResultat(resultats)

10 éléments
schéma:  ['nom', 'naissance', 'auteurDbpedia']


,nom,naissance,auteurDbpedia
0,Vickie Gendreau,1989,http://fr.dbpedia.org/resource/Vickie_Gendreau
1,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
2,Geneviève Castrée,1981,http://fr.dbpedia.org/resource/Genevi%C3%A8ve_Castr%C3%A9e
3,Isabelle Périer,1978,http://fr.dbpedia.org/resource/Isabelle_P%C3%A9rier
4,Nathanaël Dupré La Tour,1977,http://fr.dbpedia.org/resource/Nathana%C3%ABl_Dupr%C3%A9_La_Tour
5,Coralie Delaume,1976,http://fr.dbpedia.org/resource/Coralie_Delaume
6,Claire Guézengar,1972,http://fr.dbpedia.org/resource/Claire_Guezengar
7,Philippe Nassif,1971,http://fr.dbpedia.org/resource/Philippe_Nassif
8,Fatou Cissé,1971,http://fr.dbpedia.org/resource/Fatou_Fanny-Ciss%C3%A9
9,Benoït Violier,1971,http://fr.dbpedia.org/resource/Beno%C3%AEt_Violier


### BNF A3 : Les auteurs d'une oeuvre philosophique

Voir T1 pour la liste des thèmes possibles autres que la philosophie.

On se limite aux oeuvres écrite en français et sont l'auteur est référencé dans dbpedia france.

In [11]:
requete = BNF_prefix + """
SELECT distinct ?nom ?auteurDbpedia ?type_ouvrage
WHERE {

  ?livre dc:creator ?auteur .

  # un livre est un ouvrage textuel
  # (revoir la formulation de ce triplet...)
  # ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a pour theme "Philosophie"
  ?livre bnf:subject "Philosophie" .

  # le nom de l'auteur
  ?auteur foaf:name ?nom .

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER ( regex(?livreWiki, "http://wikidata.org")
       &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
ORDER BY ?auteurDbpedia

offset 0 LIMIT 20
"""

BNF.setQuery(requete)
resultats =  BNF.query().bindings
afficherResultat(resultats, 5)

20 éléments
schéma:  ['nom', 'auteurDbpedia']


,nom,auteurDbpedia
0,Alexis Carrel,http://fr.dbpedia.org/resource/Alexis_Carrel
1,Antoine de Rivarol,http://fr.dbpedia.org/resource/Antoine_de_Rivarol
2,Bernard-Henri Lévy,http://fr.dbpedia.org/resource/Bernard-Henri_L%C3%A9vy
3,Bernard de Fontenelle,http://fr.dbpedia.org/resource/Bernard_Le_Bouyer_de_Fontenelle
4,Claude Guillon,http://fr.dbpedia.org/resource/Claude_Guillon_%28%C3%A9crivain%29


###  BNF L1 : Les livres avec leurs auteurs

Les livres écrit en français au 21eme siècle,  référencés dans wikidata et dont l'auteur est référencé dans dbpedia

In [12]:
requete = BNF_prefix + """
SELECT distinct ?annee ?nom ?titre ?auteurDbpedia
WHERE {

  ?livre dc:creator ?auteur .

  # l'année de parution du livre
  ?livre dc:date ?annee .

  # le titre du livre
  ?livre dc:title ?titre .

  # un livre est un ouvrage textuel
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le nom de l'auteur
  ?auteur foaf:name ?nom.

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER (
       xsd:integer (?annee) > '2000'^^xsd:integer
    && xsd:integer (?annee) < '2099'^^xsd:integer
    && regex(?livreWiki, "http://wikidata.org")
    &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
#ORDER BY desc(?annee)

LIMIT 20
"""

BNF.setQuery(requete)
resultats =  BNF.query().bindings
afficherResultat(resultats)

20 éléments
schéma:  ['annee', 'nom', 'titre', 'auteurDbpedia']


,annee,nom,titre,auteurDbpedia
0,2006,Jonathan Littell,Les Bienveillantes,http://fr.dbpedia.org/resource/Jonathan_Littell
1,2001,Claude Simon,Le tramway,http://fr.dbpedia.org/resource/Claude_Simon
2,2002,André Gide,Le ramier,http://fr.dbpedia.org/resource/Andr%C3%A9_Gide
3,2001,Yves Bonnefoy,Les planches courbes,http://fr.dbpedia.org/resource/Yves_Bonnefoy
4,2003,J. M. G. Le Clézio,Révolutions,http://fr.dbpedia.org/resource/J._M._G._Le_Cl%C3%A9zio
5,2001,Catherine Millet,La vie sexuelle de Catherine M.,http://fr.dbpedia.org/resource/Catherine_Millet
6,2001,Michel Houellebecq,Plateforme,http://fr.dbpedia.org/resource/Michel_Houellebecq
7,2005,Michel Onfray,Traité d'athéologie,http://fr.dbpedia.org/resource/Michel_Onfray
8,2006,Pascal Quignard,Villa Amalia,http://fr.dbpedia.org/resource/Pascal_Quignard
9,2007,Nathacha Appanah,Le dernier frère,http://fr.dbpedia.org/resource/Nathacha_Appanah


## Source 2 : DBPedia France

### D1 : Informations sur une personne dont le nom contient certains mots.
Afficher le nom, nationalité, profession, date et lieu de naissance

In [13]:
requete_parametree = DBPEDIA_prefix + """
  SELECT ?p ?name ?nationalité ?profession year(xsd:dateTime(?naissance)) as ?naissance  ?lieu
  WHERE {
    ?p a dbo:Person .
    ?p foaf:name ?name .
    ?p dbo:birthDate ?naissance
    optional {?p dbp:nationalité ?nationalité}
    optional {?p dbp:profession ?profession}
    optional {?p dbp:lieuDeNaissance ?lieu}

  FILTER (regex(?p, "{MOTIF}", "i") )
  }
  LIMIT 10
"""

# l'expression régulière du nom à retrouver
motif = "Victor.*Hug"

requete = requete_parametree.replace("{MOTIF}", motif)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['p', 'name', 'nationalité', 'profession', 'naissance', 'lieu']


,p,name,nationalité,profession,naissance,lieu
0,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
1,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
2,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
3,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
4,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
5,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
6,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
7,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
8,http://fr.dbpedia.org/resource/Victor_Hugo_Green,Victor Hugo Green,Américain,,1892,http://fr.dbpedia.org/resource/New_York
9,http://fr.dbpedia.org/resource/Victor_Hugo_Green,Victor Hugo Green,Américain,,1892,http://fr.dbpedia.org/resource/États-Unis


### D2 : les infos détaillées sur une certaine personne

In [20]:
requete_parametree = DBPEDIA_prefix + """
  SELECT distinct  ?name ?nationalite ?interet ?idee
  WHERE {
    ?pers foaf:name ?name
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
  }
  order by ?interet ?idee
  LIMIT 20
"""
# On fixe une certaine personne
personne = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'
#personne = 'http://fr.dbpedia.org/resource/Judith_Butler'

# fixer le nom de la personne dans la requête
requete = requete_parametree.replace("{PERSONNE}", personne)

# print("la requête est :", requete)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

3 éléments
schéma:  ['name', 'interet', 'idee']


,name,interet,idee
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Philosophie,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Politique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Éthique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"


### D3 : la liste des oeuvres principales d'une personne donnée

In [17]:
requete_parametree = DBPEDIA_prefix +  """
    SELECT distinct ?name ?oeuvre
    WHERE {
      ?pers foaf:name ?name
      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
}
order by ?oeuvre
LIMIT 10
"""
# On fixe une certaine personne
personne = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'

requete = requete_parametree.replace("{PERSONNE}", personne)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

5 éléments
schéma:  ['name', 'oeuvre']


,name,oeuvre
0,Simone de Beauvoir,L'Invitée
1,Simone de Beauvoir,La Force de l'âge
2,Simone de Beauvoir,Le Deuxième Sexe
3,Simone de Beauvoir,Les Mandarins
4,Simone de Beauvoir,Mémoires d'une jeune fille rangée


### D4 : les influenceurs d'une personne donnée

In [18]:
requete_parametree = DBPEDIA_prefix + """
   SELECT distinct ?name ?influenceur
    WHERE {
      ?pers foaf:name ?name
      optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
}
order by ?influenceur
LIMIT 10
"""

# On fixe une certaine personne
PERSONNE = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'
#PERSONNE = 'http://fr.dbpedia.org/resource/Judith_Butler'

# fixer le nom de la personne dans la requête
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
print(len(res), 'resultats')
afficherResultat(res)

10 resultats
10 éléments
schéma:  ['name', 'influenceur']


,name,influenceur
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Alfred_Adler
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Edmund_Husserl
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Emmanuel_Kant
3,Simone de Beauvoir,http://fr.dbpedia.org/resource/François_Poullain_de_La_Barre
4,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Engels
5,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Nietzsche
6,Simone de Beauvoir,http://fr.dbpedia.org/resource/Georg_Wilhelm_Friedrich_Hegel
7,Simone de Beauvoir,http://fr.dbpedia.org/resource/Jean-Paul_Sartre
8,Simone de Beauvoir,http://fr.dbpedia.org/resource/Karl_Marx
9,Simone de Beauvoir,http://fr.dbpedia.org/resource/Martin_Heidegger


### D5 : Les auteurs nés en 2001
Utiliser la fonction year pour extraire l'année de naissance à partir d'une date de naissance.

In [21]:
requete_parametree = DBPEDIA_prefix + """
    SELECT distinct ?p ?name year(xsd:dateTime(?naissance)) as ?naissance
    WHERE {
      ?p a dbo:Person .
      ?p foaf:name ?name .
      ?p dbo:birthDate ?naissance
      FILTER( regex(?naissance, "{ANNEE}") && STRLEN(?name) != 0)
    }
    order by ?p
    LIMIT 10
"""

année = 2001
requete = requete_parametree.replace("{ANNEE}", str(année))

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['p', 'name', 'naissance']


,p,name,naissance
0,http://fr.dbpedia.org/resource/Abraham_Attah,Abraham Attah,2001
1,http://fr.dbpedia.org/resource/Adam_Siao_Him_Fa,Adam Siao Him Fa,2001
2,http://fr.dbpedia.org/resource/Adrien_Warion,Adrien Warion,2001
3,http://fr.dbpedia.org/resource/Adèle_Castillon,Adèle Castillon,2001
4,http://fr.dbpedia.org/resource/Ai_Ogura,Ai Ogura,2001
5,http://fr.dbpedia.org/resource/Aiko_de_Toshi,Aiko,2001
6,http://fr.dbpedia.org/resource/Aleksandr_Nikichine,Aleksandr Nikichine,2001
7,http://fr.dbpedia.org/resource/Aleksandr_Smolyar,Aleksandr Smolyar,2001
8,http://fr.dbpedia.org/resource/Aleksej_Pokuševski,Aleksej Pokuševski,2001
9,http://fr.dbpedia.org/resource/Alessandro_Michieletto,Alessandro Michieletto,2001


### D6 : La liste de tous les intérets principaux


In [22]:
requete = DBPEDIA_prefix + """
    SELECT distinct ?interet
    WHERE {
      ?personne dbo:mainInterest ?o .
      ?o rdfs:label ?interet .

      FILTER (langMatches( lang(?interet), "FR" ) )
}
order by ?interet
LIMIT 10
"""

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['interet']


,interet
0,Activisme politique
1,Afrique
2,Agnosticisme
3,Alchimie
4,Amour
5,Amour platonique
6,Angélologie
7,Anthropologie
8,Anthropologie linguistique
9,Anthropologie philosophique


### D7 la liste des philosophes dans Dbpedia

In [23]:
import time

requete = DBPEDIA_prefix + """

SELECT distinct  ?name ?p
WHERE {
  ?p rdf:type dbo:Philosopher .
  ?p foaf:name ?name
}
order by ?name
limit 10
"""

start_time = time.time()

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings

duree = int ( (time.time() - start_time) * 1000)
print(f"durée {duree} ms")

afficherResultat(res)


durée 26167 ms
10 éléments
schéma:  ['name', 'p']


,name,p
0,,http://fr.dbpedia.org/resource/Œuvre_de_Friedrich_Nietzsche
1,,http://fr.dbpedia.org/resource/Abu_'Ali_Al-Jubbâ'i
2,,http://fr.dbpedia.org/resource/Abu_al-Hudhayl
3,,http://fr.dbpedia.org/resource/Abū_Hāshīm_al-Jubbā'ī
4,,http://fr.dbpedia.org/resource/Adam_Smith
5,,http://fr.dbpedia.org/resource/Aimé_Forest
6,,http://fr.dbpedia.org/resource/Al-Fârâbî
7,,http://fr.dbpedia.org/resource/Al-Ghazali
8,,http://fr.dbpedia.org/resource/Alain_(philosophe)
9,,http://fr.dbpedia.org/resource/Alfred_Tarski


# Exercice 1 : Requêtes réparties entre les 2 sources BNF et DBPedia
A compléter

### J1: Jointure A3 avec D2 par boucles imbriquées
Proposer une solution pour calculer la jointure entre A3 sur le site BNF et D2 sur le site DBPedia.
Pour chaque livre de afficher son titre et son année, le nom de l'auteur, sa nationalité et la liste de ses idées remarquables.

In [32]:
# A compléter
requete_BNF = BNF_prefix + """
SELECT distinct ?nom ?auteurDbpedia ?type_ouvrage
WHERE {

  ?livre dc:creator ?auteur .

  # un livre est un ouvrage textuel
  # (revoir la formulation de ce triplet...)
  # ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a pour theme "Philosophie"
  ?livre bnf:subject "Philosophie" .

  # le nom de l'auteur
  ?auteur foaf:name ?nom .

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER ( regex(?livreWiki, "http://wikidata.org")
       &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
ORDER BY ?auteurDbpedia

LIMIT 20
"""

requete_DBPEDIA_parametree = DBPEDIA_prefix + """
  SELECT distinct  ?name ?nationalite ?interet ?idee
  WHERE {
    ?pers foaf:name ?name
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
  }
  order by ?interet ?idee
  LIMIT 20
"""

BNF.setQuery(requete_BNF)
resultats_BNF = BNF.query().bindings

resultat_jointure = []
for r1 in tableauResultat(resultats_BNF) :
    personne = r1[1]
    requete_DBPEDIA = requete_DBPEDIA_parametree.replace("{PERSONNE}", personne)
    DBPEDIA.setQuery(requete_DBPEDIA)
    resultats_DBPEDIA = DBPEDIA.query().bindings
    for r2 in tableauResultat(resultats_DBPEDIA) :
      join = r1 + r2[1:]
      resultat_jointure.append(join)






In [33]:
print(len(resultat_jointure))

33


In [34]:
for r in resultat_jointure :
    print(r)

['Emmanuel Levinas', 'http://fr.dbpedia.org/resource/Emmanuel_Levinas', 'http://fr.dbpedia.org/resource/Histoire', "Autre/Autrui, Altérité, Visage, Événement, Éros, Au-delà de l'être, Illéité"]
['Emmanuel Levinas', 'http://fr.dbpedia.org/resource/Emmanuel_Levinas', 'http://fr.dbpedia.org/resource/Religion', "Autre/Autrui, Altérité, Visage, Événement, Éros, Au-delà de l'être, Illéité"]
['Emmanuel Levinas', 'http://fr.dbpedia.org/resource/Emmanuel_Levinas', 'http://fr.dbpedia.org/resource/Théologie', "Autre/Autrui, Altérité, Visage, Événement, Éros, Au-delà de l'être, Illéité"]
['Emmanuel Levinas', 'http://fr.dbpedia.org/resource/Emmanuel_Levinas', 'http://fr.dbpedia.org/resource/Érotisme', "Autre/Autrui, Altérité, Visage, Événement, Éros, Au-delà de l'être, Illéité"]
['Emmanuel Levinas', 'http://fr.dbpedia.org/resource/Emmanuel_Levinas', 'http://fr.dbpedia.org/resource/Éthique', "Autre/Autrui, Altérité, Visage, Événement, Éros, Au-delà de l'être, Illéité"]
['Gottfried Wilhelm Leibniz', 

### J2 : Pour une année N afficher la liste des personnes qui sont nées l'année N et qui existent à la fois dans la BNF et dans DBPedia.
Proposer une solution par fusion de deux résultats triés

In [ ]:
# A compléter

### J3 : Quels sont les auteurs d'une oeuvre dont le thème est philosophie (dans la BNF) et qui sont référencés comme étant des philosophes dans DBpedia ?

In [ ]:
# A compléter

# Exercice 2 : Accès aux données par des requêtes de sélections
L'objectif de cet exercice est d'appréhender la situation où l'accès aux données est contraint (voir la notion de [linked data fragment](https://linkeddatafragments.org/concept/)). Le point d'accès n'autorise que des requêtes simples (donc peut coûteuses à traiter) qui ne contiennent que des **sélections**. La clause where d'une requête ne peut contenir qu'**un seul** motif ayant 1 ou 2 variables. Les motifs autorisés sont donc parmi :
*   ?a *propriété* ?b *(cette requête a deux variables)*
*   *A* *propriété* ?b  *(cette requête a une seule variable)*
*   ?a *propriété* *B*  

où ?a et ?b sont des variables et *A*, *B* et *propriété* sont des valeurs fixées avant de poser la requête.

Ainsi, toutes les jointures doivent être exprimées par le client qui accède aux données : vous devez "programmer" toutes les jointures en python.
Pour les requêtes D1 et D6 proposer une solution respectant ce mode d'accès.

In [ ]:
# à compléter

# Exercice 3 : Accès par morceau au résultat d'une requête
1) Un utilisateur veut accéder aux données de manière progressive. Proposer une solution pour obtenir le resultat d'une requete par morceau de N tuples, en utilisant les mots-clés OFFSET et LIMIT. Appliquer cela pour lire le résultat de la requête qui affiche le nom des personnes existant dans dbpedia.
Etudier l'influence de l'offset et de la taille du résultat sur le temps de réponse de la requête.



In [ ]:
# Exemple d'utilisation de OFFSET et LIMIT

# Compléter cet exemple


import time

requete_parametree = DBPEDIA_prefix + """

SELECT distinct ?p ?name
WHERE {
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
  FILTER(  STRLEN(?name) != 0)
}
# ORDER BY ?p ?name
offset {nb_offset} limit 10
"""
duree_moy = 0
for i in range(10000 , 11001 , 100 ) :
  requete = requete_parametree.replace("{nb_offset}", str(i))
  start_time = time.time()
  # exécution de la requête
  DBPEDIA.setQuery(requete)
  res = DBPEDIA.query().bindings
  duree_moy += int ( (time.time() - start_time) * 1000)

print(duree_moy/11)

292.6363636363636


In [ ]:
duree_moy = 0
for i in range(0 , 101 , 10 ) :
  requete = requete_parametree.replace("{nb_offset}", str(i))
  start_time = time.time()
  # exécution de la requête
  DBPEDIA.setQuery(requete)
  res = DBPEDIA.query().bindings
  duree_moy += int ( (time.time() - start_time) * 1000)

print(duree_moy/11)

104.9090909090909


2) Définir une fonction traiteRequete(requete) qui retourne un generateur sur une requete de telle sorte que la requete soit lue par morceaux de N tuples et que seuls les morceaux permettant de produire les M lignes demandées par l'utlisateur soit lus. On ne connait pas à l'avance le nombre M de lignes que va lire l'utilisateur. On a M > N.
Indication voir le mot clé python **yield** pour retourner un générateur.

Limit N
M lignes demandées par l'utilisateur

In [ ]:

def traiteRequete(requete_parametree , N ):
  offset = 0
  while(True):
    requete = requete_parametree.replace("{nb_offset}", str(offset))
    requete = requete.replace("{N}", str(N))
    DBPEDIA.setQuery(requete)
    res = DBPEDIA.query().bindings
    print(f"lecture d'un morceau de {len(res)} lignes")
    for r in res :
      yield r
    offset += N

requete_parametree = DBPEDIA_prefix + """
SELECT distinct ?name ?p
WHERE {
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
  FILTER(  STRLEN(?name) != 0)
}
offset {nb_offset} limit {N}
"""

# Nombre de tuples que l'utilisateur souhaite lire
a_lire = 22
N = 5

lu = 0

for a in traiteRequete(requete_parametree , N):
  print(a)
  lu += 1
  if lu >= a_lire :
    break

print()
if(lu < a_lire):
  print(f"Attention !!!! l'utilisateur n'a obtenu que {lu} lignes sur les {a_lire}")
else:
  print(f"OK, l'utilisateur a obtenu {lu} lignes sur les {a_lire}")

lecture d'un morceau de 5 lignes
{'name': Value(literal:'Łukasz Bujko'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Bujko')}
{'name': Value(literal:'Łukasz Gibała'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Gibała')}
{'name': Value(literal:'Gierak'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Gierak')}
{'name': Value(literal:'Łukasz Grzeszczuk'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Grzeszczuk')}
{'name': Value(literal:'Łukasz Kaczmarek'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Kaczmarek')}
lecture d'un morceau de 5 lignes
{'name': Value(literal:'Łukasz Klekot'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Klekot')}
{'name': Value(literal:'Łukasz Kohut'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Kohut')}
{'name': Value(literal:'Łukasz Koszarek'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Koszarek')}
{'name': Value(literal:'Łukasz Krupiński'), 'p': Value(uri:'http://fr.dbpedia.org/resou

3) Proposer une solution pour accéder de manière progressive au résultat d'une requête de jointure entre 2 sites, par exemple J1.

In [51]:
requete_BNF_parametree = BNF_prefix + """
SELECT distinct ?nom ?auteurDbpedia ?type_ouvrage
WHERE {

  ?livre dc:creator ?auteur .

  # un livre est un ouvrage textuel
  # (revoir la formulation de ce triplet...)
  # ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a pour theme "Philosophie"
  ?livre bnf:subject "Philosophie" .

  # le nom de l'auteur
  ?auteur foaf:name ?nom .

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER ( regex(?livreWiki, "http://wikidata.org")
       &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
ORDER BY ?auteurDbpedia

offset {nb_offset} limit {N}
"""

requete_DBPEDIA_parametree = DBPEDIA_prefix + """
  SELECT distinct  ?name ?nationalite ?interet ?idee
  WHERE {
    ?pers foaf:name ?name
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
  }
  order by ?interet ?idee

  offset {nb_offset} limit {N}
"""


def traiteRequete2(requete_parametree , N , site):
  offset = 0
  while(True):
    requete = requete_parametree.replace("{nb_offset}", str(offset))
    requete = requete.replace("{N}", str(N))
    if site == "DBPEDIA":
      DBPEDIA.setQuery(requete)
      res = DBPEDIA.query().bindings
    else :
      BNF.setQuery(requete)
      res = BNF.query().bindings
    print(f"lecture d'un morceau de {len(res)} lignes")
    if site == "DBPEDIA" and len(res) == 0 :
      break
    for r in tableauResultat(res) :
      yield r
    offset += N

def traiteRequete_jointure(requete_BNF_parametree , requete_DBPEDIA_parametree ,  N) :
  for r1 in traiteRequete2(requete_BNF_parametree , N , "BNF"):
      print(r1[0])
      personne = r1[1]
      requete_DBPEDIA = requete_DBPEDIA_parametree.replace("{PERSONNE}", personne)
      for r2 in traiteRequete2(requete_DBPEDIA , N , "DBPEDIA"):
          join = r1 + r2[1:]
          print(join)
          yield join






# Nombre de tuples que l'utilisateur souhaite lire
a_lire = 40
N = 5

lu = 0

for a in traiteRequete_jointure(requete_BNF_parametree , requete_DBPEDIA_parametree ,  N):
  #print(a)
  lu += 1
  if lu >= a_lire :
    break

print()
if(lu < a_lire):
  print(f"Attention !!!! l'utilisateur n'a obtenu que {lu} lignes sur les {a_lire}")
else:
  print(f"OK, l'utilisateur a obtenu {lu} lignes sur les {a_lire}")


lecture d'un morceau de 5 lignes
Alexis Carrel
lecture d'un morceau de 0 lignes
Antoine de Rivarol
lecture d'un morceau de 0 lignes
Bernard-Henri Lévy
lecture d'un morceau de 0 lignes
Bernard de Fontenelle
lecture d'un morceau de 0 lignes
Claude Guillon
lecture d'un morceau de 0 lignes
lecture d'un morceau de 5 lignes
Denis Diderot
lecture d'un morceau de 0 lignes
Emer de Vattel
lecture d'un morceau de 0 lignes
Emmanuel Levinas
lecture d'un morceau de 5 lignes
['Emmanuel Levinas', 'http://fr.dbpedia.org/resource/Emmanuel_Levinas', 'http://fr.dbpedia.org/resource/Histoire', "Autre/Autrui, Altérité, Visage, Événement, Éros, Au-delà de l'être, Illéité"]
['Emmanuel Levinas', 'http://fr.dbpedia.org/resource/Emmanuel_Levinas', 'http://fr.dbpedia.org/resource/Religion', "Autre/Autrui, Altérité, Visage, Événement, Éros, Au-delà de l'être, Illéité"]
['Emmanuel Levinas', 'http://fr.dbpedia.org/resource/Emmanuel_Levinas', 'http://fr.dbpedia.org/resource/Théologie', "Autre/Autrui, Altérité, Visage

# Exerice 4 : Cache de requêtes
1) Proposer des améliorations des requêtes précédentes afin d'éviter de poser plusieurs fois une même requête. On suppose que le résultat de la requête ne change pas entre deux invocations de la même requête.

2) Cache avec une capacité limitée : considérer qu'on peut conserver en cache le résultat de C requêtes (C=10) au maximum

In [ ]:
# A compléter

# FAQ: Questions diverses


## Mise au point des requêtes
Pour débugguer et **mettre au point vos requêtes** Sparql, il est utile de les tester directement sur l'interface du point d'accès dédié de la source de données. L'avantage est que cela vous retourne un message indiquant très précisément la nature et l'endroit où se trouve l'erreur dans la requête.
*   [BNF](http://data.bnf.fr/sparql) voir des [exemples](https://api.bnf.fr/sparql-endpoint-de-databnffr)
*   [DBPedia](http://fr.dbpedia.org/sparql) et la liste de tous les [prefixes](http://fr.dbpedia.org/sparql?nsdecl)


## Autres sources de données
Autres sources à votre disposition pour proposer des requêtes multi-sources:

1.   [Mondial](http://www.semwebtech.org/mondial/10/sparql) : données géopolitiques. Attention le résulat des requêtes est en XML pas en JSON.
2.   [Persée](data.persee.fr) : données bibliographiques alignées avec plusieurs autres sources dont dbpedia. Voir des [exemples](http://data.persee.fr/ressources/requetes-exemples/)
3.   [BioPortal](http://sparql.bioontology.org)
4.    [IdRef](https://data.idref.fr/sparql) serveur d’autorité IdRef et les références bibliographiques en provenance du Sudoc.
5.   [Insee](https://rdf.insee.fr/sparql)
6.  [Geonames web services](https://www.geonames.org/export/ws-overview.html)


## SPARKLIS : aide à l'écriture de requêtes Sparql
Pour exprimer vos requêtes en *langage naturel* et les traduire automatiquement en SPARQL, voir l'appli [Sparklis](http://www.irisa.fr/LIS/ferre/sparklis/)


*   onglet **Sparklis** pour exprimer vos requêtes en langage naturel
*   onglet **YASGUI** pour voir la traduction en Sparql



## Nombre d'éléments dans le resultat d'une requête
Pour compter le nombre d'éléments d'une requête, utiliser la notation avec requête imbriquée dans le where :

SELECT (count(1) as ?nb)

WHERE { *sous-requete* }
